1. Implement Apriori algorithm in Python. Use the transaction data which you used in data mining as the input.

Source: Pg. 287 of "Data Mining - Concepts and Techniques" by Jiawei Han, Micheline Kamber, and Jian Pei (3rd Edition - Morgan Kaufmann - 2011)

In [1]:
TID = ['T100', 'T200', 'T300', 'T400', 'T500', 'T600', 'T700', 'T800', 'T900']
transactions = [['I1', 'I2', 'I5'], ['I2', 'I4'], ['I2', 'I3'], ['I1', 'I2', 'I4'], ['I1', 'I3'], ['I2', 'I3'], ['I1', 'I3'], ['I1', 'I2', 'I3', 'I5'], ['I1', 'I2', 'I3']]
min_support = 2

data = dict(zip(TID, transactions))

In [2]:
data

{'T100': ['I1', 'I2', 'I5'],
 'T200': ['I2', 'I4'],
 'T300': ['I2', 'I3'],
 'T400': ['I1', 'I2', 'I4'],
 'T500': ['I1', 'I3'],
 'T600': ['I2', 'I3'],
 'T700': ['I1', 'I3'],
 'T800': ['I1', 'I2', 'I3', 'I5'],
 'T900': ['I1', 'I2', 'I3']}

In [3]:
# Convert all itemsets to 'frozensets'

for i in data.keys():
    data[i] = frozenset(data[i])

In [4]:
data

{'T100': frozenset({'I1', 'I2', 'I5'}),
 'T200': frozenset({'I2', 'I4'}),
 'T300': frozenset({'I2', 'I3'}),
 'T400': frozenset({'I1', 'I2', 'I4'}),
 'T500': frozenset({'I1', 'I3'}),
 'T600': frozenset({'I2', 'I3'}),
 'T700': frozenset({'I1', 'I3'}),
 'T800': frozenset({'I1', 'I2', 'I3', 'I5'}),
 'T900': frozenset({'I1', 'I2', 'I3'})}

In [5]:
# Generate First / Initial Candidate Itemsets

def UniqueItems(data):
    unique_items = set()
    
    for i in data.values():
        unique_items = unique_items.union(i)
    
    return unique_items

In [6]:
c1 = UniqueItems(data)
c1

{'I1', 'I2', 'I3', 'I4', 'I5'}

In [7]:
def CalculateSupport(itemsets, data):
    # Candidate Itemsets [C]
    c = {}
    
    for i in itemsets:
        c[i] = 0
        for transaction in data.values():
            if type(i) != frozenset:
                c[i] += list(transaction).count(i)
            else:
                if set(i).issubset(transaction):
                    c[i]+=1
    
    return c

# If 'I1 I2 I3 I1 I2 I3' occurs in an itemset,
# This algorithm must be extended to handle that.
# As of now, it can only handle 'I1 I2 I3'
# That is, it can only handle non-repetitive items in itemsets

In [8]:
def PruneItemset(c, min_support):
    infrequent_itemsets = []

    for itemset in c:
        if c[itemset] < min_support:
            infrequent_itemsets.append(itemset)

    for itemset in infrequent_itemsets:
        c.pop(itemset)

    return c

In [9]:
c1 = CalculateSupport(c1, data)
l1 = PruneItemset(c1, min_support)

l1

{'I3': 6, 'I4': 2, 'I2': 7, 'I5': 2, 'I1': 6}

In [10]:
def JoinOperation(c, k=2):
    if k < 2:
        return c
    
    elif k == 2:
        # Non-duplicate Combinations of all elements
        c = [frozenset([i, j]) for i in c for j in c if i != j]
        c = set(c)
        return c   
    
    else:
        c = list(c)
        c.sort()
        
        new_c = []
        for i in c:
            tmp = list(i)
            tmp.sort()
            new_c.append(tmp)
            
        out = set()
        for i in new_c:
            for j in new_c:
                # 'k-2' elements must be same to join
                # 'k' -> number of items in itemset to be created
                if i != j and i[:k-2] == j[:k-2]:
                    out.add(frozenset(i).union(frozenset(j)))
        return out

In [11]:
c2 = JoinOperation(c1, k=2)
c2 = CalculateSupport(c2, data)
l2 = PruneItemset(c2, min_support)

l2

{frozenset({'I1', 'I3'}): 4,
 frozenset({'I2', 'I3'}): 4,
 frozenset({'I2', 'I4'}): 2,
 frozenset({'I2', 'I5'}): 2,
 frozenset({'I1', 'I2'}): 4,
 frozenset({'I1', 'I5'}): 2}

In [12]:
c3 = JoinOperation(c2, k=3)
c3 = CalculateSupport(c3, data)
l3 = PruneItemset(c3, min_support)

l3

{frozenset({'I1', 'I2', 'I5'}): 2, frozenset({'I1', 'I2', 'I3'}): 2}

In [13]:
c4 = JoinOperation(c3, k=4)
c4 = CalculateSupport(c4, data)
l4 = PruneItemset(c4, min_support)

l4

{}

In [14]:
def DisplayItemsets(itemsets):
    keys = list(itemsets.keys())
    
    new_keys = []
    for key in keys:
        # If the itemset has only a single item
        if type(key) != frozenset:
            new_keys.append(key)
        # If the itemset has multiple items
        else:
            # Sort the items in itemset
            tmp = list(key)
            tmp.sort()
            new_keys.append(tmp)
            
    # Sort the entire list of itemsets
    new_keys.sort()
    
    for key in new_keys:
        if type(key) != list:
            print(f"{key} :\t{itemsets[key]}")
        else:
            tmp = list(key)
            tmp.sort()
            print(f"{tmp} :\t{itemsets[frozenset(key)]}")
    print()

In [15]:
def AprioriAlgorithm(data, min_support):
    tmp = UniqueItems(data)
    
    i = 2
    while True:
        c = CalculateSupport(tmp, data)
        c = PruneItemset(c, min_support)
        
        print(f"*** L{i-1} ***")
        DisplayItemsets(c)
        
        tmp = c
        tmp = JoinOperation(tmp, k=i)
        
        if len(c) == 0:
            break
        else:
            i+=1

In [16]:
AprioriAlgorithm(data, min_support=2)

*** L1 ***
I1 :	6
I2 :	7
I3 :	6
I4 :	2
I5 :	2

*** L2 ***
['I1', 'I2'] :	4
['I1', 'I3'] :	4
['I1', 'I5'] :	2
['I2', 'I3'] :	4
['I2', 'I4'] :	2
['I2', 'I5'] :	2

*** L3 ***
['I1', 'I2', 'I3'] :	2
['I1', 'I2', 'I5'] :	2

*** L4 ***

